In [1]:
import pandas as pd
from db_queries import get_location_metadata
from pathlib import Path


## Get data

#### Get BMI data

In [132]:
BMI_FILEPATH = '/mnt/team/integrated_analytics/pub/goalkeepers/goalkeepers_2024/data/bmi/bmi_data_outliered.csv'
bmi_raw_df = pd.read_csv(BMI_filepath,dtype={'hh_id': str})


/tmp/ipykernel_3974133/3891060890.py:2: DtypeWarning: Columns (46,47,48,53) have mixed types. Specify dtype option on import or set low_memory=False.
  bmi_raw_df = pd.read_csv(BMI_filepath,dtype={'hh_id': str})


In [79]:
# BMI wealth factor is all NA :( 
# Gotta bring the wealth data in from the other team
bmi_raw_df.wealth_factor.isna().all()

True

### Get Wealth Data and transform to income

In [4]:
def get_income_from_asset_score(asset_df:pd.DataFrame,  asset_score_col= 'asset_score', year_df_col = 'year_start'):
    #Get and clean income data
    INCOME_FILEPATH = '/mnt/team/rapidresponse/pub/population/data/02-processed-data/cgf_bmi/income_distributions.parquet'
    income_raw = pd.read_parquet(INCOME_FILEPATH)
    income_raw = income_raw[['pop_percent', 'cdf', 'location_id', 'year_id']]
    income_raw['location_id'] = income_raw.location_id.astype(int)
    income_raw['year_id'] = income_raw.year_id.astype(int)

    wdf = asset_df.copy()
    
    # We get which asset score is what percentile of that asset score for each nid
    get_percentile = lambda x: x.rank() / len(x) 

    assert('percentile' not in wdf.columns)
    assert('nid' in wdf.columns)
    assert('location_id' in wdf.columns)
    wdf['percentile'] = wdf.groupby(['nid'], group_keys=False)[asset_score_col].apply(get_percentile)

    wdf = wdf.sort_values(['percentile'])
    income_raw = income_raw.sort_values(['pop_percent'])

    income_interp = income_raw.merge(
        wdf[['location_id', year_df_col, 'percentile']].drop_duplicates().rename(columns={year_df_col:'year_id', 'percentile':'pop_percent'}),
        on=['location_id', 'year_id', 'pop_percent'],
        how='outer')

    # We then interpolate to get the percentiles that aren't included in the income dataset
    income_interp = income_interp.sort_values(['location_id', 'year_id', 'pop_percent'])
    income_interp = income_interp.set_index(['location_id', 'year_id', 'pop_percent'])
    income_interp['income_per_day'] = income_interp.groupby(level=[0, 1], group_keys=False)['cdf'].apply(lambda x: x.interpolate(method='linear', limit_area='inside'))
    income_interp = income_interp.reset_index()

    wdf = wdf.merge(income_interp, how='left', left_on=['location_id', 'year_start', 'percentile'],
        right_on = ['location_id', 'year_id', 'pop_percent'])

    # Only NAs should be because of newer surveys for which we don't have income distribution yet
    assert((wdf.loc[wdf.income_per_day.isna()].year_id > 2020).all())
    return wdf

In [165]:
WEALTH_FILEPATH = '/mnt/share/scratch/users/victorvt/cgfwealth_spatial/dhs_wealth_uncollapsed_again.parquet'
loc_meta = get_location_metadata(release_id = 9, location_set_id = 35)

wealth_raw = pd.read_parquet(WEALTH_FILEPATH)
wealth_df = wealth_raw#[wealth_raw['point'] == 1]
wealth_df = wealth_df.rename(columns = {'wealth_score':'asset_score'})
wealth_df = wealth_df[['iso3', 'nid', 'psu', 'hh_id', 'year_start', 'asset_score']].drop_duplicates()
# In the wealth team's dataset, sometimes there are multiple asset scores for a given household id.
# Take away those NIDs
bad_wealth = wealth_df.groupby(['nid', 'hh_id', 'year_start', 'psu',]).size()
bad_nid_wealth = list(bad_wealth[bad_wealth.gt(1)].reset_index().nid.unique())
bad_nid_wealth = bad_nid_wealth + [20315]
wealth_df = wealth_df[~wealth_df.nid.isin(bad_nid_wealth)]
# Make sure that by nid, psu and hh_id they all have the same lat and long
#grouped = wealth_df.groupby(['nid', 'psu', 'hh_id'])
#assert((grouped['lat'].nunique().lt(2) & grouped['long'].nunique().lt(2)).all())
# Sometimes an nid has more than a year
assert(wealth_df.groupby(['nid', 'hh_id', 'year_start', 'psu']).size().sort_values().max() == 1)
wealth_df = wealth_df.merge(loc_meta[['location_id', 'local_id']], left_on ='iso3', right_on='local_id', how='left')
assert(wealth_df.location_id.notna().all())
wealth_df['year_start'] = wealth_df['year_start'].astype(int)
wealth_df['nid'] = wealth_df['nid'].astype(int)

wealth_df = get_income_from_asset_score(wealth_df)


#### Get Climate information

In [160]:
CLIMATE_FILEPATH = '/mnt/team/rapidresponse/pub/population/data/02-processed-data/cgf_bmi/bmi_climate.parquet'

#clim_raw = pd.read_parquet(CLIMATE_FILEPATH)

window_size_for_last_years_avg = 5

climate_df = pd.read_parquet(CLIMATE_FILEPATH)
climate_df = climate_df.rename(columns={'latnum':'lat', 'longnum':'long'})
climate_df = climate_df.sort_values(['year', 'lat', 'long'])
rolling_mean_window_f = lambda x: x.rolling(window=window_size_for_last_years_avg).mean()
cumulative_mean_f = lambda x: x.shift().expanding().mean()
climate_df['over30_cumavg'] = climate_df.groupby(['lat', 'long'])['over30'].transform(cumulative_mean_f)
climate_df['temp_cumavg'] = climate_df.groupby(['lat', 'long'])['temp'].transform(cumulative_mean_f)
climate_df['precip_cumavg'] = climate_df.groupby(['lat', 'long'])['precip'].transform(cumulative_mean_f)
climate_df['over30_diff'] = climate_df['over30'] - climate_df['over30_cumavg']
climate_df['temp_diff'] = climate_df['temp'] - climate_df['temp_cumavg']
climate_cols = [col for col in climate_df.columns if col not in ['lat', 'long', 'year']]

# I don't think it makes sense to do the "Mean # of days over 30 in lifetime" since we probably don't have enough years of info
# And it probably only matters when you're young. 

#### Trim and merge

In [166]:
# Trim to only NIDs in common between BMI and wealth data
common_nids = set(bmi_raw_df['nid'].unique()).intersection(set(wealth_df['nid'].unique()))
wealth_df = wealth_df[wealth_df['nid'].isin(common_nids)]
bmi_df = bmi_raw_df[bmi_raw_df['nid'].isin(common_nids)].copy()

# Drop unused columns
wealth_df = wealth_df[['iso3', 'nid', 'psu', 'hh_id', 'asset_score', 'income_per_day' ]]
bmi_df = bmi_df[['nid', 'psu', 'hh_id', 'latnum', 'longnum', 'sex_id', 'age_year', 'ihme_loc_id', 'year_start', 'year_end', 'urban',
    'pweight', 'bmi', 'low_adult_bmi']]
bmi_df = bmi_df.rename(columns = {'latnum':'lat', 'longnum':'long', 'hh_id':'old_hh_id'})
wealth_df = wealth_df.rename(columns = {'hh_id':'old_hh_id'})

# Try to make household id usable to merge on - 
# TODO: why is it so different between the datasets if it's the same source?
bmi_df['hh_id'] = bmi_df['old_hh_id'].str.split(r'[_ ]').str[-1]
wealth_df['hh_id'] = wealth_df['old_hh_id'].str.split(r'[_ ]').str[-1]

merge_cols = ['nid', 'hh_id', 'psu']

merged_df = bmi_df.merge(wealth_df, on=merge_cols, how='left')
print(len(bmi_df))
print(len(merged_df))

print(len(merged_df))
merged_df = merged_df[~merged_df.nid.isin(merged_df.loc[merged_df.income_per_day.isna()].nid.unique())]
print(len(merged_df))
merged_df = merged_df.drop(columns=['old_hh_id_y', 'old_hh_id_x'])
merged_df = merged_df.merge(climate_df, left_on=['lat', 'long', 'year_start'],
    right_on=['lat', 'long', 'year'], how='left')
print(len(merged_df))
assert(merged_df[merged_df.over30.isna()].year_start.gt(2016).all())
merged_df = merged_df[merged_df.year_start < 2017]
print(len(merged_df))
 

1547221
1547221
1547221
1495037
1495037
1465676


In [177]:
OUT_ROOT = Path("/mnt/team/rapidresponse/pub/population/data/02-processed-data/cgf_bmi")
merged_df.to_parquet(OUT_ROOT / "bmi_processed.parquet")